In [18]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
# preprocessing nlp
m = ['mf','mb','habit','wsls','kdh']
df_theoretical_and_lr_fit = []
df_rnn_fit = []
h = []
b = []

for model in m:
    
    # theoretical and lr nlp
    df = pd.read_csv(f'../../results/{model}/{model}_fit.csv')
    df = df.groupby(np.arange(len(df))//5).mean()
    select_cols = [col for col in df.columns if 'test_nlp' in col]
    df = df[select_cols]
    df_theoretical_and_lr_fit.append(df)
    
    # rnn nlp
    df = pd.read_csv(f'../../results/{model}/rnn_fit_{model}.csv')
    df = df.groupby(np.arange(len(df))//5).mean()
    df = pd.DataFrame(df['test_nlp_rnn'].values.reshape(-1,100).T,
             columns=['test_nlp_rnn_100','test_nlp_rnn_200',
                      'test_nlp_rnn_400','test_nlp_rnn_600',
                      'test_nlp_rnn_1000'])
    cc = df.columns
    for index,row in df.iterrows():
        c = cc[row.argmin()].split('_')[-1]
        h.append(c)
        b.append(row.min())
    df_rnn_fit.append(df)

df_0 = pd.concat(df_theoretical_and_lr_fit)
df_0.reset_index(inplace=True)
df_0['agent'] = np.arange(500)
df_0['index'] = np.repeat(['mf','mb','habit','wsls','kdh'],100)

df_0.rename(columns={'index':'model'},inplace=True)  
df_0 = df_0.reindex(columns=['agent','model', 'test_nlp_mf', 'test_nlp_mb', 'test_nlp_habit',
                       'test_nlp_wsls','test_nlp_kdh', 'test_nlp_logistic_regression'])

df_1 = pd.concat(df_rnn_fit)
df_1.reset_index(inplace=True)
df_1['best_test_nlp_rnn'] = b
df_1['best_epoch'] = h
df_1['agent'] = np.arange(500)
df_1['index'] = np.repeat(['mf','mb','habit','wsls','kdh'],100)
df_1.rename(columns={'index':'model'},inplace=True)

df_0 = df_0.merge(df_1)

df_0.to_csv('all_model_nlp.csv')